1.树的建立

    给定一个二维空间数据集，T=[(2,3),(5,4),(9,6),(4,7),(8,1),(7,2)],构造一个平衡kd树    

    思路引导：
    根节点对应包含数据集T的矩形，选择x(1)轴，6各数据点的x(1)坐标中位树是6，这里选择最接近的(7, 2)点，以平面x(1)=7将空间分为左、右两个子矩形（子节点）；
    接着左矩形以x(2)=4分为两个子矩形（左矩形中{(2,3)(5,4)(4,7)}点的x(2)坐标中位数
    正好是4），右矩形以x(2)=6分为两个子矩形，如此递归，最后得到特征空间划分和kd树

![jupyter](../Sources/Pictures/k-近邻算法/kd树构建2.png)

    1.数据分析：
        第一维度：2，5，9，4，8，7
        第二维度：3，4，6，7，1，2
    2.确定根节点
        （1）确定划分维度：选择方差较大（分散程度越高）那个维度进行划分
                第一维度：2-9     第二维度：1-7
            所以第一维度的方差更大，划分第一维度
        （2）对第一维度进行排序：2，4，5，7，8，9
        （3）确定根节点：a.确定中位数：6        （5+7）/2=6
                      b.确定根节点（选取距离中位数较近的点）
                            （5，4）中第一维度5，与（7，2）中第一维度7距离6都相等
                            任意选择，此处选择（5，4）
                         即：根节点（7，2）
        （4）以根节点（7，2）为标准，并以第一维度中的7为标准，将数据划分
                第一维度比7小的：2，4，5
                第一维度比7大的：8，9 
    3.递归迭代划分子节点
        （1）跟换维度：以第二维度为标准进行划分，同时分别计算所有两个节点的中位数，确定子节点
                左侧第二维度：2，4，5---->中位数：4---->对应坐标：(5,4)
                右侧第二维度：8，9---->由于只有两个数随便选一个，如果有多个数据，那么就要求中位数---->9---->对应坐标：（9，6）
        （2）kd树前前两层出来了：
                                            （7，2）
                                    （5，4）          （9，6）
        （3）同理得到kd树：
             第一层                               （7，2）
             第二层                    （5，4）               （9，6）
             第三层              （2，3）     （4，7）         （8，1） 

![jupyter](../Sources/Pictures/k-近邻算法/kd树构建1.png)

    2.最近邻域的搜索
       假设标记为星星的点是test point，绿色的点是找到的近似点，在回溯过程中，需要用到
       一个队列，存储需要有回溯的点，在判断其他子节点空间中是否有可能有距离查询点更近的
       数据点时，做法是以查询点为圆心，以当前的而最近距离为半径画圆，这个圆称为超球（
       candidate hypersphere），如果圆与回溯点的轴相交，则需要将轴另一边的节点都放
       到回溯队列里面 。

![jupyter](../Sources/Pictures/k-近邻算法/最近邻域搜索.png)

        2.1 查找点（2.1，3.1）
            （1）在（7，2）点测试到大（5，4），在（5，4）点测试道德（2，3），然后
                search_path中的节点为<(7,2),(5,4),(2,3)>，从search_path中
                取出作为当前最佳结点nearset，dist为0.141
            （2）然后回溯至（5，4），以（2.1，3.1）为圆心，以dist=0.141为半径的
                圆，未不和超平面y=4相交，如上图，所以不必跳到结点（5，4）的右子空
                间去搜索，因为右子空间不可能有更接近样本点的了。
            （3）于是再回溯至（7，2），同理，以（2.1，3.1）为圆心，以dist=0.141
                为半径画一个圆并不和超平面x=7相交，所以也不用跳到节点（7，2）的右
                子空间去搜索。
            （4）至此，search_path为空，结果整个搜索，返回nearest（2.1，3.1）
                的最近邻点，最近距离为0.141

![jupyter](../Sources/Pictures/k-近邻算法/案例1.png)

        2.2 查找点（2，4.5）
            （1）在（7，2）处测试到达（5，4），在（5，4）处测试到达（4，7）[优先选
                择在本域搜索]。然后search_path中的结点为<(7,2),(5,4),(4,7)>
                ，从search_path中取出当前最佳节点nearest，dist为3.202
            （2）然后回溯至（5，4），以（2，4.5）为圆心，以dist=3.202为半径画一
                个圆与超平面y=4相交，所以需要跳到（5，4）的左子空间去搜索。所以要
                将（2，3）加入到search_path中，现在search_path中的结点为
                <(7,2),(2,3)>;另外，（5，4）与（2，4.5）的距离为3.04 <  
                dist=3.202，所以将（5，4）赋值给nearest，并且dist=3.04
            （3）回溯至（2，3），（2，3）是叶子节点，直接评判（2，3）是哦否离（2，
                4.5）更近，计算得到距离为1.5，所以nearest更新为（2，3），dist
                更新为（1，5）.
            （4）回溯至（7，2），同理，以（2，4.5）为圆心，以dist=1.5为半径画一
                个圆，并不是和超平面x=7相交，所以不用跳到结点（7，2）的右子空间去
                搜索。
            （5）至此，search_path为空结束整个搜索，返回nearest（2，3）作为（2
                ，4.5）的最近邻点，最近距离为1.5

![jupyter](../Sources/Pictures/k-近邻算法/案例2.png)